In [ ]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

#nettoyer le corpus
import jieba

#modèle LDA
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

#visualiser
import pyLDAvis
import pyLDAvis.gensim_models

#calculer les distances
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist, squareform

In [ ]:
#étape 1 : préparer les données

#importer le fichier excel et extraires les informations de la colonne inscriptions
data = pd.read_excel("fr_insc.xlsx")
data.fillna("",inplace=True)
texte = list(data["inscriptions"])

#importer les listes des stopwords
jieba.load_userdict("dict_txt_big.txt")
with open('stopwords1.txt', encoding = 'UTF-8') as f:
    stop_words = f.readlines()

#nettoyer les stopwords et supprimant les caractères indésirables
stop_words = [w.replace('\n', '') for w in stop_words]
stop_words = [w.replace(' ', '') for w in stop_words]

#supprimer les mots vides, les ponctuations, les alphabets, les chiffres et separer
rule = re.compile(r"[^\u4e00-\u9fa5\u3400-\u4dbf]") #regex pour ne garder que les caractères chinois
texte = [list(jieba.cut(rule.sub('', i))) for i in texte] #couper le texte en liste de phrases
for idx, t in enumerate(texte):
    texte[idx] = ' '.join([word for word in t if word.strip() not in stop_words]) #supprimer les mots vides

In [ ]:
#couper de nouveau la liste de phrases en liste de mots
result_fenci = [i.split(' ') for i in texte]
v = [sous_liste for sous_liste in result_fenci if all(mot != '' for mot in sous_liste)]

In [ ]:
#étape 2 : stocker les données dans un dictionnaire gensim
dictionary = gensim.corpora.Dictionary(v)
corpus_final = [dictionary.doc2bow(doc) for doc in v]

In [ ]:
#étape 3 : entrainer le modèle
#exécuter le lda, LdaModel pour les données plus petites. Générer 5 topics, passer 100 fois
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_final,
                id2word=dictionary,
                num_topics=5,
                passes=100,
                )

In [ ]:
#évaluer
#calculer la cohérence
coherence_model_lda = CoherenceModel(model=lda_model, texts=v, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Cohérence: ', coherence_lda)

#calculer la perplexité
print('Perplexité: ', lda_model.log_perplexity(corpus_final)

In [ ]:
#étape 4 : générer les topics obtenus
topics = [] #liste vide pour stocker les thèmes
for idx, topic in lda_model.print_topics(-1) : #parcourir tous les mots pour les ajouter à la liste
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)

all_topic_model = [] #liste vide pour stocker les modèles
for i in range(len(topics)): #parcourir tous les mots de la liste topics
  str = topics[i].split(' + ')
  topic_model = [] #liste vide pour enregistrer :
  for j in range(10):
    weight = str[j][0:5] #les poids
    word = str[j][7:len(str[j])-1] #les mots
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)

#stocker le résultat dans un dataframe
df_topic_model = pd.DataFrame(all_topic_model)
df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 2: "Topic 3", 3: "Topic 4", 4: "Topic 5", 5: "Topic 6", 6: "Topic 7", 7: "Topic 8", 8: "Topic 9", 9: "Topic 10"})


In [ ]:
#exporter le dataframe
df_topic_model.to_excel('lda_t5_tableau.xlsx')

In [ ]:
#étape 5 : visualiser
pyLDAvis.enable_notebook()
vis_data = pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)
#exporter au format HTML
pyLDAvis.save_html(vis_data, 'lda_t5.html')

In [ ]:
#étape 6 : distance et dendrogramme
#obtenir la distribution des topics
topic_dist = lda_model.state.get_lambda()

In [ ]:
#calculer la distance euclidienne
dist = pd.DataFrame(squareform(pdist(topic_dist), 'euclidean'))

In [ ]:
#créer un dendrogramme
cluster_names = ['T1', 'T2', 'T3', 'T4', 'T5']
#cluster_names = ['T1', 'T2', 'T3', 'T4', 'T5','T6', 'T7', 'T8', 'T9', 'T10']
dendrogramme = shc.dendrogram(shc.linkage(dist),  labels=cluster_names)